<a href="https://colab.research.google.com/github/Aminolroaya/Langchain_for_Chat_with_YouTube_Video/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install required packages
! pip install langchain
! pip install pypdf
!pip install openai
!pip install python-dotenv
!pip install tiktoken
! pip install chromadb
!pip install lark
! pip install yt_dlp
! pip install pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 94.5 MB/s e

In [18]:
#read from Youtube
import openai
import sys
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

openai.api_key =  "use your api key ;)"
url="https://www.youtube.com/watch?v=duJk42smCqk"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=duJk42smCqk
[youtube] duJk42smCqk: Downloading webpage
[youtube] duJk42smCqk: Downloading ios player API JSON
[youtube] duJk42smCqk: Downloading android player API JSON
[youtube] duJk42smCqk: Downloading m3u8 information
[info] duJk42smCqk: Downloading 1 format(s): 140
[download] docs/youtube//Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a has already been downloaded
[download] 100% of    8.41MiB
[ExtractAudio] Not converting audio docs/youtube//Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a; file is already in target format m4a
Transcribing part 1!


In [34]:
# split the document
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=400,
    chunk_overlap=4,
    length_function=len
)
splits = text_splitter.split_documents(docs)

In [35]:
# save in vector DB
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
#embeddingSent1 = embedding.embed_query("life is short, so enjoy it")
#embeddingSent2 = embedding.embed_query("Be weird!")
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory='docs/chroma/'
)
#question = "what the course is about?"
#docs = vectordb.similarity_search(question,k=5)
#for doc in docs:
#    print(doc.page_content)

In [42]:
question = "What is the major topic for this video?"
#docs = vectordb.similarity_search(question,k=3)
print(vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3))
print(vectordb.similarity_search("will the prices decrease?",k=3))

[Document(page_content='Hello there and welcome to my channel, Living in Calgary, Alberta', metadata={'chunk': 0, 'source': 'docs/youtube/Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a'}), Document(page_content="That way you get updated on the very next video. Alright, so let's get started. I'm going to go into some of the graphs and charts that our real estate board provides us, and this gives us a better indication and kind of look at what the trend is and what's causing the prices to go up", metadata={'chunk': 0, 'source': 'docs/youtube/Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a'})]
[Document(page_content="So, as you can see, low inventory usually means prices are on the rise. And as you can see over here as well, the detached new listings are down, and for the last, since 2021, each year this time, they're just continually going down. So, at some point, this will probably change. You know, no trend lasts forever", metadata={'chunk':

In [48]:
#Load LLM and Retrival Q&A using LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=openai.api_key)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": question})
print(result)
result = qa_chain({"query": "Will the home prices decrease?"})
print(result)

{'query': 'What is the major topic for this video?', 'result': 'The major topic for this video is the Calgary real estate market update for July 2023, specifically discussing whether Calgary house prices will be dropping in 2023.'}
{'query': 'Will the home prices decrease?', 'result': 'Based on the given context, there is no clear indication that home prices in Calgary will decrease in the near future. The speaker mentions that prices have been rising and there is a trend of decreasing inventory for detached homes. However, they also mention that no trend lasts forever, so it is possible that prices could change in the future. It is important to note that this information is based on the July 1st real estate market update and may not reflect future market conditions.'}


In [49]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [53]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({"query": "what you dont know?"})
print(result)

{'query': 'what you dont know?', 'result': "I don't know what specific graphs and charts the real estate board provides. Thanks for asking!", 'source_documents': [Document(page_content="That way you get updated on the very next video. Alright, so let's get started. I'm going to go into some of the graphs and charts that our real estate board provides us, and this gives us a better indication and kind of look at what the trend is and what's causing the prices to go up", metadata={'chunk': 0, 'source': 'docs/youtube/Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a'}), Document(page_content="That's one of the things that some people think is that realtors set the price, and it's actually the market that sets the price. It's a supply and demand rule or a law that dictates the value of anything, and houses are no different. So, as long as there is a high demand and low supply, people are willing to pay more for a house. So, you know, people need to live somewhere", meta

In [59]:
#have a memory, take history and the new question
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
result = qa_chain({"query": "Will the home prices decrease?"})
print(result)
result = qa_chain({"query": "can you give me a summary of the video?"})
print(result)

{'query': 'Will the home prices decrease?', 'result': 'It is unclear if home prices will decrease based on the given information. Thanks for asking!', 'source_documents': [Document(page_content="So, as you can see, low inventory usually means prices are on the rise. And as you can see over here as well, the detached new listings are down, and for the last, since 2021, each year this time, they're just continually going down. So, at some point, this will probably change. You know, no trend lasts forever", metadata={'chunk': 0, 'source': 'docs/youtube/Are House Prices In Calgary Dropping？ The Facts May Surprise You!.m4a'}), Document(page_content="So, that's this bottom left graph here. The blue bar is the inventory, the green bar is sales, and the red trend line is the benchmark price. And as you can see, June last year, we saw actually the detached homes fall in value. It wasn't a lot, but it did fall a bit, and as you can see, the distance between the inventory and sales is quite a bit